<a href="https://colab.research.google.com/github/k1151msarandega/QuCode-21-Days-of-Quantum-Challenge-Diary/blob/main/Day06_Dirac_Notation_%26_Hilbert_Spaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 06 — Dirac Notation & Hilbert Spaces

> _QuCode 21 Days of Quantum Challenge — Learning notebook_
>
> **Date:** 2025-09-06  
> **Author:** Kudzai Musarandega  
> **Tags:** quantum, learning, challenge, day-06
>
> **Learning objectives**
> - See **bras** as linear functionals (dual vectors) and use **bra–ket** notation fluently.
> - Move between **kets** (state vectors), **bras** (row conjugates), and **inner / outer** products.
> - Work with **basis states**, matrix elements `⟨i|A|j⟩`, and the **resolution of identity**.
> - Interpret **operators**: expectation values, projectors, changes of basis, and simple measurements.
>
>
> **Key takeaways (summary-first)**
> - A **ket** $|\psi\rangle$ lives in a (complex) Hilbert space $\mathcal H$.  
>  Its **bra** is $ \langle\psi| \equiv |\psi\rangle^\dagger$ (conjugate transpose): a linear functional $\mathcal H\to\mathbb C$.
> - **Inner product:** $\langle\phi|\psi\rangle$ (linear in the right slot, anti-linear in the left).  
>   **Outer product:** $|\psi\rangle\langle\phi|$ (rank-1 operator).
> - **Resolution of identity** in an orthonormal basis $\{|i\rangle\}$:  
>  $$\sum_i |i\rangle\langle i|=I\quad\Rightarrow\quad |\psi\rangle=\sum_i |i\rangle\langle i|\psi\rangle.$$
> - **Matrix elements:** $A_{ij}=\langle i|A|j\rangle$. Expectation: $\langle A\rangle_\psi=\langle\psi|A|\psi\rangle$.
> - **Projector** onto $|\psi\rangle$ (normalized): $P_\psi=|\psi\rangle\langle\psi|$, with $P_\psi^2=P_\psi$, $P_\psi^\dagger=P_\psi$.
> - Discrete (finite-dim) vs **continuous** bases: $\sum_i\to\int \mathrm d x$, $\delta_{ij}\to\delta(x-x')$.



## Resources
- **Official/Assigned:**
    - [Quantum Sense: What are bras and bra-ket notation? | Maths of Quantum Mechanics](https://www.youtube.com/watch?v=lRR-qgjaKlg)
    - [Alexander (fufaev.org): Bra-Ket Notation and How to Use It](https://www.youtube.com/watch?v=mAZSmzv_asU)
    - [For the Love of Physics: What is Dirac Notation? Kets, Bras, Inner Products & Operators](https://www.youtube.com/watch?v=I6wn0TxR2aA)
- **Extra reading:**
    - [Faculty of Khan: Mathematical Basis of Quantum Mechanics](https://youtube.com/playlist?list=PLdgVBOaXkb9Bv466YnyxslT4gIlSZdtjw&si=1qCgx0fNrzvOJUTE)
- **Original notes:**


In [ ]:
# %% [markdown]
# ### Environment setup (Colab)
# If you are running on Colab for the first time today, uncomment to install.
# This cell intentionally avoids heavy installs by default.
#
# !pip -q install qiskit pennylane matplotlib numpy

import sys, platform, math, json, numpy as np

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
np.random.seed(42)


## 1) Little bra–ket helper toolkit (NumPy)

In [1]:
## 1) Little bra–ket helper toolkit (NumPy)
# - ket: column vector
# - bra: conjugate transpose
# - inner/outer, expectation, projectors, basis utilities

import numpy as np

def ket(vec):  # ensure column shape
    v = np.asarray(vec, dtype=complex).reshape(-1)
    return v

def bra(v):    # ⟨v| = |v⟩†
    return ket(v).conj().T

def inner(phi, psi):      # ⟨phi|psi⟩
    return np.vdot(phi, psi)  # np.vdot does conj(phi)·psi

def outer(psi, phi):      # |psi⟩⟨phi|
    return np.outer(ket(psi), ket(phi).conj())

def proj(psi):            # |psi⟩⟨psi| (assumes normalized)
    return outer(psi, psi)

def norm(psi):
    return np.linalg.norm(psi)

def normalize(psi):
    n = norm(psi)
    return psi / n if n != 0 else psi

def expect(A, psi):       # ⟨ψ|A|ψ⟩
    ψ = ket(psi)
    return bra(ψ) @ (A @ ψ)

def is_projector(P, tol=1e-10):
    return (np.allclose(P, P.conj().T, atol=tol)
            and np.allclose(P @ P, P, atol=tol))

## 2) Basis states, inner vs outer products, and resolution of identity

In [3]:
## 2) Basis states, inner vs outer products, and resolution of identity

# Choose a 3D toy space to see sums work out cleanly
e0 = ket([1,0,0])
e1 = ket([0,1,0])
e2 = ket([0,0,1])
ONB = [e0, e1, e2]  # orthonormal basis (computational)

# Resolution of identity: ∑ |i><i| = I
I3 = sum(outer(v, v) for v in ONB)
print("Resolution of identity holds?", np.allclose(I3, np.eye(3)))

# Expand a state by inserting I = ∑ |i><i|
psi = normalize(ket([2, 1-1j, 0.5j]))
coeffs = [inner(v, psi) for v in ONB]       # coefficients ⟨i|ψ⟩
reconstructed = sum(c * v for c, v in zip(coeffs, ONB))
print("Reconstruction ok?", np.allclose(psi, reconstructed))

# Inner vs outer: one is scalar, the other is operator
phi = normalize(ket([0, 1, 1j]))
print("⟨φ|ψ⟩ =", inner(phi, psi))           # complex scalar
print("|ψ⟩⟨φ| shape =", outer(psi, phi).shape)


Resolution of identity holds? True
Reconstruction ok? True
⟨φ|ψ⟩ = (0.4242640687119285-0.282842712474619j)
|ψ⟩⟨φ| shape = (3, 3)


## 3) Projectors as measurements & overlaps (Born rule)

In [4]:
## 3) Projectors as measurements & overlaps (Born rule)
# P_ψ projects any |φ⟩ onto span{|ψ⟩}; probability of "ψ" when in |φ⟩ is |⟨ψ|φ⟩|^2

Pψ = proj(psi)
print("Pψ is a projector?", is_projector(Pψ))

# Apply Pψ to |φ⟩ and compare with overlap scalar
φ_proj = Pψ @ phi
amp = inner(psi, phi)      # ⟨ψ|φ⟩
print("Projection equals amp*|ψ⟩ ?", np.allclose(φ_proj, amp*psi))

prob = abs(amp)**2
print("Born probability |⟨ψ|φ⟩|^2 =", prob.real)


Pψ is a projector? True
Projection equals amp*|ψ⟩ ? True
Born probability |⟨ψ|φ⟩|^2 = 0.26


## 4) Operators in a basis: matrix elements and expectation values

In [5]:
## 4) Operators in a basis: matrix elements and expectation values
# Build a Hermitian operator A from its spectral decomposition, check ⟨ψ|A|ψ⟩

# Orthonormal eigenbasis { |i⟩ } with eigenvalues λ_i
lmbdas = np.array([2.0, -1.0, 0.5])
A = sum(lmbdas[i] * outer(ONB[i], ONB[i]) for i in range(3))  # A = ∑ λ_i |i⟩⟨i|
print("Hermitian?", np.allclose(A, A.conj().T))

# Matrix elements ⟨i|A|j⟩ in ONB
A_mat = np.array([[inner(ONB[i], A @ ONB[j]) for j in range(3)] for i in range(3)])
print("Matrix of A in ONB =\n", A_mat)

# Expectation in |ψ⟩ (two equivalent ways)
exp1 = expect(A, psi)
exp2 = sum(lmbdas[i] * abs(inner(ONB[i], psi))**2 for i in range(3))  # spectral weights
print("⟨A⟩ via matrix =", exp1, " ; via spectrum =", exp2)


Hermitian? True
Matrix of A in ONB =
 [[ 2. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j -1. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0.5+0.j]]
⟨A⟩ via matrix = (0.9800000000000001+0j)  ; via spectrum = 0.9800000000000002


## 5) Change of basis via a unitary (columns are new basis kets)

In [8]:
## 5) Change of basis via a unitary (columns are new basis kets)

rng = np.random.default_rng(42)
M = rng.normal(size=(3,3)) + 1j*rng.normal(size=(3,3))
Q, _ = np.linalg.qr(M)    # Q is unitary; use it as the basis-change matrix
U = Q

# New basis kets (columns of U)
tilde_basis = [U @ e0, U @ e1, U @ e2]
I_new = sum(outer(v, v) for v in tilde_basis)
print("New basis orthonormal & complete?", np.allclose(I_new, np.eye(3)))

# Coordinates of |ψ⟩ in old vs new basis
coords_old = np.array([inner(e, psi) for e in ONB])             # ⟨i|ψ⟩
coords_new = np.array([inner(v, psi) for v in tilde_basis])     # ⟨i~|ψ⟩

# Relationship: coords_new = U† coords_old
coords_new_from_U = U.conj().T @ coords_old     # or: dag(U) @ coords_old
print("Coefficient transform ok?", np.allclose(coords_new, coords_new_from_U))


New basis orthonormal & complete? True
Coefficient transform ok? True


## 6) Bra–ket algebra cheatsheet (discrete, orthonormal basis)
- Linearity: ⟨φ| (a|ψ⟩ + b|χ⟩) = a⟨φ|ψ⟩ + b⟨φ|χ⟩ ; (a⟨φ| + b⟨χ|) |ψ⟩ = a*⟨φ|ψ⟩ + b*⟨χ|ψ⟩
- Conjugate symmetry: ⟨ψ|φ⟩ = ⟨φ|ψ⟩*
- Resolution: ∑_i |i⟩⟨i| = I
- Components: |ψ⟩ = ∑_i |i⟩ ⟨i|ψ⟩ ;   A = ∑_{ij} |i⟩⟨i|A|j⟩⟨j|
- Projector: P_ψ = |ψ⟩⟨ψ| (idempotent, Hermitian); measurement prob = |⟨ψ|φ⟩|^2
- Expectation: ⟨A⟩_ψ = ⟨ψ|A|ψ⟩ ; Variance: ⟨A^2⟩_ψ - ⟨A⟩_ψ^2


## 7) Continuous vs discrete (heads-up for later)
For a continuous basis $\{|x\rangle\}$:
- Completeness: $\displaystyle \int |x\rangle\langle x|\,\mathrm dx = I$
- Orthogonality: $\langle x|x'\rangle=\delta(x-x')$
- Wavefunction: $\psi(x)=\langle x|\psi\rangle$, and inner product $\displaystyle \langle\phi|\psi\rangle=\int \phi^\*(x)\psi(x)\,\mathrm dx$

> In code you approximate integrals by sums on a grid (discretisation). Same algebra, with $\sum \Delta x \approx \int$.


In [10]:
## (7a) Discretising a wavefunction (continuous → grid)
# Inner product ⟨φ|ψ⟩ ≈ Σ φ*(x_k) ψ(x_k) Δx

import numpy as np

# grid
x = np.linspace(-8.0, 8.0, 4001)
dx = x[1] - x[0]

def gauss(x, x0=0.0, sigma=1.0, k=0.0):
    norm = (1.0/(np.pi*sigma**2))**0.25
    return norm * np.exp(-0.5*((x - x0)/sigma)**2) * np.exp(1j*k*x)

# two (normalized) wavefunctions
psi_x = gauss(x, x0=-1.0, sigma=1.2, k=0.6)
phi_x = gauss(x, x0=+0.5, sigma=0.9, k=-0.2)

# numeric inner product (Riemann sum)
inner_num = np.vdot(phi_x, psi_x) * dx          # np.vdot does conjugate on first arg
print("⟨φ|ψ⟩ (numeric) =", inner_num)
print("|⟨φ|ψ⟩| ≤ 1 ?", abs(inner_num) <= 1 + 1e-12)

# check normalization on grid
norm_psi = np.vdot(psi_x, psi_x) * dx
norm_phi = np.vdot(phi_x, phi_x) * dx
print("‖ψ‖² ≈", norm_psi.real, "  ‖φ‖² ≈", norm_phi.real)


⟨φ|ψ⟩ (numeric) = (0.5031781057061367-0.016107197681433157j)
|⟨φ|ψ⟩| ≤ 1 ? True
‖ψ‖² ≈ 0.9999999999998895   ‖φ‖² ≈ 0.9999999999998894


## 8) Try it yourself
1) **Matrix elements:** For a random Hermitian $H$, verify $H=\sum_{ij}|i\rangle\langle i|H|j\rangle\langle j|$ numerically.  
2) **Two-outcome measurement:** Given normalized $|\psi\rangle$ and a normalized $|\phi\rangle$, build projectors $P_\phi$ and $I-P_\phi$.  
   Simulate one Born measurement (random number $r\in[0,1)$): collapse to $|\phi\rangle$ if $r<|\langle\phi|\psi\rangle|^2$, else to the orthogonal subspace.  
3) **Change of basis:** Create a unitary $U$ (QR) and verify that $\sum_i |i^\sim\rangle\langle i^\sim|=I$ where $|i^\sim\rangle=U|i\rangle$.  
4) **Expectation vs sampling:** For diagonal $A=\mathrm{diag}(\lambda_0,\lambda_1,\lambda_2)$, draw $10^4$ outcomes by sampling $i$ with probabilities $|\langle i|\psi\rangle|^2$ and compare the empirical mean with $\langle A\rangle_\psi$.


## 9) Reflection
- How does viewing **bras** as linear functionals (dual vectors) make identities like $\sum_i |i\rangle\langle i|=I$ feel “obvious”?  
- Why is the **outer product** the right tool for building projectors and changing bases?  
- In what ways do these identities simplify algorithm derivations (e.g., inserting $I$ to prove circuit identities)?


Optional figures you can add

Dual space sketch: column vector (ket) vs conjugate row (bra) acting on it to produce a scalar.

Resolution of identity: tiles showing $|i\rangle\langle i|$ blocks summing to $I$.

Projector geometry: vector $|\phi\rangle$ decomposed into component along $|\psi\rangle$ plus orthogonal remainder.

---
### Links
- **Open in Colab (from GitHub):** replace `YOUR_GITHUB_USERNAME/qucode-21days`
  - `https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/qucode-21days/blob/main/Day06_Entanglement_Basics.ipynb.ipynb`
- **Report an issue / suggest a fix:** link to your repo issues page
